# AILab 画像分類コンペティション

## ルール
検証にCIFAR10のValデータは使わないでください。

(今回評価用のデータとして用いるため)

モデルを評価する際は、Trainデータを分割して使ってください。

In [ ]:
#ライブラリのインポート
import torch as torch
import torchvision
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm
from torchsummary import summary
from pylab import rcParams

from utils import make_dataset
from utils import eval

### 1.1. データセットの読み込み（改変不可）
- pytorchではデータをTensor形式で扱います。(listやnumpyと似たような構造を持つ)

In [ ]:
#このセルは改変しないで
train_dataset=make_dataset()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./dataset/cifar-10-python.tar.gz to ./dataset/


### 1.2. データセットの可視化

In [ ]:
%matplotlib inline
def show(img):
    npimg = img.numpy() 
    plt.grid(False) 
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

print (train_dataset)
image, label = train_dataset[1]
print("学習画像サイズ",image.size())
print ("ラベル:",label)
show(image)

### 1.3. データセットをミニバッチ単位に変換します。
- **torch.utils.data.DataLoader**クラスで行います。

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=200, #バッチサイズの定義
                                           shuffle=True,
                                          num_workers=1)


## 2.モデルの定義
### 2.1. モデルの定義(nn.Moduleを継承したクラスでの書き方)
- **__init__**関数でネットワーク層の定義、初期化を行います。
- **forward**関数でモデルの入力→出力の定義、順伝搬の計算を行います。  
    - 定義したネットワーク層に入力(x)を伝搬していきます。
- 全結合ネットワークは**nn.Liner(入力パラメータ数、出力パラメータ数)**で定義します。
    - 画像データを扱う場合、入力パラメータ数は**二次元データを一次元に変換**して入力します。
    - Tensorのサイズ変換には**Tensor.view(軸について要素数を指定)**を用います。

In [ ]:
class atte_Block(nn.Module):
    def __init__(self, channels_in, channels_out):
        super(atte_Block, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(channels_in, channels_out, kernel_size=3, padding=1),
            nn.BatchNorm2d(channels_out),
            nn.GELU(),
        )
        self.atte = nn.Sequential(
            nn.Conv2d(channels_out, channels_out, kernel_size=3, padding=1),
            nn.GELU(),
            nn.Conv2d(channels_out, channels_out, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        h = self.features(x)
        h_atte=self.atte(h)
        y=h*h_atte
        return y

class cnn_Block(nn.Module):
    def __init__(self, inp, oup):
        super(cnn_Block, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(inp, oup, kernel_size=3, padding=1),
            nn.BatchNorm2d(oup),
            nn.GELU()
        )
    
    def forward(self, x):
        h = self.features(x)
        y = h + x
        return y
    
class atte_CNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(atte_CNN, self).__init__()
        num=32
        self.atte_block0 = atte_Block(input_dim, num) 
        self.blocks0 = nn.ModuleList([cnn_Block(num, num) for _ in range(3)])
        self.maxpool0 = nn.MaxPool2d(kernel_size=2)
        self.atte_block1 = atte_Block(num, num) 
        self.blocks1 = nn.ModuleList([cnn_Block(num, num) for _ in range(3)])
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.full = nn.Sequential(
            nn.Linear(16*16*num, 4096),
            nn.GELU(),
            nn.Linear(4096, output_dim),
        )
        
    def forward(self, x):
        h = self.atte_block0(x)
        for block in self.blocks0:
            h = block(h)
        h = self.maxpool0(h)
        h = self.atte_block1(h)
        for block in self.blocks1:
            h = block(h)
        h = self.maxpool1(h)
        h = h.view(x.size(0),-1)
        y = self.full(h)
        return y 

#print(atte_CNN(3, 10))
device = 'cuda' if torch.cuda.is_available() else 'cpu' #GPUの定義
net = atte_CNN(3, 10).to(device)

### 2.2. モデルの可視化
- summary(モデル,入力サイズ(チャンネル数、パラメータ数))クラスで可視化を行います。

In [ ]:
summary(net, input_size=(3,64,64))

## 3. モデルの学習
### 3.1. 損失関数、最適化法(オプティマイザー)の定義
- 損失関数は**nn.CrossEntropyLoss()**(ソフトマックス+交差エントロピー)を用います。
- 最適化法には**optim.SGD(適用するモデルの重み、その他ハイパーパラメータの設定・・・)**を用います。

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001,  weight_decay=5e-4)

### 3.2. 学習の流れ(エポック単位)
- ①画像データと正解ラベルを取り出す
    - tqdmはjupyter内にプログレスバー(進行度)を表示する
- ②画像データとラベルデータ(共にバッチ単位)をデバイス(GPU)に移動
- ③勾配の初期化
- ④モデルの推測(画像データ→各クラスである確率)
- ⑤損失の計算
- ⑥⑤で求めた損失をエポックでの累計損失に加える
    - Tensorから値を取得する場合は、1次元(1要素)に指定してから.item()で取得する必要がある
- ⑦正答率の計算(出力の最大値のインデックスが正解ラベルの場合1を出力をバッチサイズ分行う(.sum())
    - (.maxは()で比較する値の軸を選び、[0]に最大値の値、[1]に最大値のインデックスを持つ)
- ⑧逆伝搬の計算
- ⑨勾配の更新<br>

---全バッチで行う---
- ⑩エポックでの平均誤差の計算
- ⑪エポックでの平均正答率の計算
- 検証データでの損失、精度を学習時と同じアルゴリズムで検証(逆伝搬、勾配の更新は行わない)

*1:学習時には.train()で学習に関連する機能を有効にします。   
*2:検証時には.eval()で学習に関連する機能を有効に、torch.no_grad()で自動微分を停止してから検証を行います。

In [ ]:
num_epochs = 5 #学習エポックの設定

#グラフ作成のためにエポックごとの各値を保存する配列を作成
train_loss_list = []
train_acc_list = []


#学習の定義
for epoch in range(num_epochs):
    train_loss = 0
    train_acc = 0
    i=0#学習回数
    
    net.train() #train *1
    for (images, labels) in train_loader: #①
        images, labels = images.to(device), labels.to(device) #②
        optimizer.zero_grad() #③
        outputs = net(images) #④
        loss = criterion(outputs, labels)#⑤
        train_loss += loss.item()#⑥
        train_acc += (outputs.max(1)[1] == labels).sum().item() #⑦
        loss.backward()#⑧
        optimizer.step()#⑨
        
        
    
    avg_train_loss = train_loss / len(train_loader.dataset) #⑩
    avg_train_acc = train_acc / len(train_loader.dataset)   #⑪
    
    print('epoch *', (epoch+1))
    print('Loss: {:.4f}, acc: {:.4f}%'.format(avg_train_loss, 100*avg_train_acc))
    #グラフ壁画用に各値を配列に格納
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)

epoch * 1
Loss: 0.0147, acc: 36.9860%
epoch * 2
Loss: 0.0063, acc: 54.7860%
epoch * 3
Loss: 0.0050, acc: 64.0620%
epoch * 4
Loss: 0.0041, acc: 70.6980%
epoch * 5
Loss: 0.0034, acc: 76.0820%


## 4. 学習記録のグラフ化
- matplotlibを用います。

In [ ]:
#学習のグラフ化
plt.figure()
plt.plot(range(num_epochs), train_loss_list, color='blue', linestyle='-', label='train_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Training loss')
plt.grid()

plt.figure()
plt.plot(range(num_epochs), train_acc_list, color='blue', linestyle='-', label='train_acc')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title('Training accuracy')
plt.grid()

## 評価用セル(改変不可)

In [ ]:
score=eval(net)
print("スコア",score)

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


スコア 0.7106
